In [34]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter



In [8]:
conn = sqlite3.connect('Global_Coral_Bleaching_Database_SQLite_11_24_21.db')
if conn:
    print('success')
else:
    print('error')

success


In [106]:
query = pd.read_sql_query('''SELECT event.Sample_ID, event.Site_ID, event.Date_Day, event.Date_Month, 
                            event.Date_Year, site.Latitude_Degrees, site.Longitude_Degrees, bleach.Severity_Code, 
                            enviro.Temperature_Mean FROM Sample_Event_tbl AS event
                                JOIN Site_Info_tbl AS site ON site.Site_ID = event.Site_ID
                                JOIN Bleaching_tbl AS bleach ON event.Sample_ID = bleach.Sample_ID
                                JOIN Environmental_tbl AS enviro ON enviro.Sample_ID = bleach.Sample_ID
                                WHERE bleach.Severity_Code >= 0;
                                ''', conn)

df = pd.DataFrame(query)
df['Temp_Mean_F'] = (df['Temperature_Mean'] - 273.15) * 9/5 + 32
df.head()

,Sample_ID,Site_ID,Date_Day,Date_Month,Date_Year,Latitude_Degrees,Longitude_Degrees,Severity_Code,Temperature_Mean,Temp_Mean_F
0,9625,4608,29.0,6.0,2010.0,24.457072,124.098558,0,298.930000,78.404
1,9632,4615,2.0,6.0,2010.0,24.799825,125.178566,0,298.640000,77.882
2,9635,4618,23.0,8.0,2009.0,26.247572,127.668171,0,297.920000,76.586
3,9638,4621,18.0,7.0,2009.0,26.275176,127.714069,0,297.863333,76.484
4,9640,4623,21.0,8.0,2010.0,26.443525,127.772670,0,297.820000,76.406


In [22]:
mapbox_access_token = 'pk.eyJ1Ijoia2VuY2hhbjUzMCIsImEiOiJjbGliNzdpMmUwNHZ5M3Ruejl1OGV2NTR3In0.joFuJScQcCTs-p9nfrRYrg'

In [108]:
# apply gaussian filter
lats=np.array(df['Latitude_Degrees'])
lons=np.array(df['Longitude_Degrees'])

sigma=0.32

blur_lats = gaussian_filter(lats, sigma)
blur_lons = gaussian_filter(lons, sigma)

In [119]:
figure = go.Figure()

figure.add_trace(go.Scattermapbox(
                  lat=blur_lats,
                  lon=blur_lons,
                  customdata=np.stack((df['Severity_Code'], df['Temp_Mean_F']), axis=-1),
                  marker=go.scattermapbox.Marker(
                        size=df['Severity_Code']*5,
                        color=df['Temp_Mean_F'],
                        opacity=0.1,
                        colorscale='Turbo',
                        colorbar=dict(title='Water Temperature (F)'),
                  ),
                  hovertemplate='<br>'.join([
                   'Severity Code: %{customdata[0]}',
                   'Mean Temp(F): %{customdata[1]:.2f}'   
                  ]),
                  name=''
))

figure.update_layout(
              title='Locations of Coral Bleaching',
              mapbox=dict(
                    accesstoken=mapbox_access_token,
                    bearing=0,
                    pitch=0,
                    zoom=0.5,
                    style='basic',
                    ),
              height=500,
              width=900,
              )

figure.show()
